In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.stats as ss
import yt
import time
import scipy.ndimage as sd

In [3]:
# List of all primary variable files obtained after the simulation run
# Grid = 1200 Mm x 1200 Mm x 400 Mm 

nx=1200
ny=1200
nz=400

# Magnetic field
#bz = np.fromfile('/home/nylab1/data/mhd_50G/result_prim_5.072400',dtype=np.float32).reshape(nx,ny,nz)
#bx = np.fromfile('/home/nylab1/data/mhd_50G/result_prim_6.072400',dtype=np.float32).reshape(nx,ny,nz)
#by = np.fromfile('/home/nylab1/data/mhd_50G/result_prim_7.072400',dtype=np.float32).reshape(nx,ny,nz)

# Plasma velocity
vz = np.fromfile('/home/nylab1/data/mhd_50G/result_prim_1.072400',dtype=np.float32).reshape(nx,ny,nz)
vx = np.fromfile('/home/nylab1/data/mhd_50G/result_prim_2.072400',dtype=np.float32).reshape(nx,ny,nz)
vy = np.fromfile('/home/nylab1/data/mhd_50G/result_prim_3.072400',dtype=np.float32).reshape(nx,ny,nz)

# Mass density, Temperature, Pressure
#m = np.fromfile('/home/nylab1/data/mhd_50G/result_prim_0.072400',dtype=np.float32).reshape(nx,ny,nz)
#T = np.fromfile('/home/nylab1/data/mhd_50G/eosT.072400',dtype=np.float32).reshape(nx,ny,nz)
#P = np.fromfile('/home/nylab1/data/mhd_50G/eosP.072400',dtype=np.float32).reshape(nx,ny,nz)

In [ ]:
# All three velocity components
%matplotlib inline

comp = ["x","y","z"]
vel = [vx,vy,vz]
fig, ax = plt.subplots(1,3,figsize=(25,7),sharey=True)

for i in range(len(comp)):
    ax[i].set(title=str(comp[i])+" component")
    ax[i].set(title=str(comp[i])+" component")
    figure=ax[i].imshow(np.rot90(vel[i][:,:,150]),cmap='CMRmap', aspect='auto', extent=[0.,12., 0., 12.])
    clb=fig.colorbar(figure,ax=ax[i])
    ax[i].set_xlim(0,12)
    ax[i].set_ylim(0,12)

In [ ]:
# All three magnetic field components

comp = ["x","y","z"]
B = [bx,by,bz]
fig, ax = plt.subplots(1,3,figsize=(25,7),sharey=True)

for i in range(len(comp)):
    ax[i].set(title=str(comp[i])+" component")
    ax[i].set(title=str(comp[i])+" component")
    figure=ax[i].imshow(B[i][:,:,150],cmap='CMRmap', aspect='auto', extent=[0.,12., 0., 12.])
    clb=fig.colorbar(figure,ax=ax[i])
    ax[i].set_xlim(0,12)
    ax[i].set_ylim(0,12)

In [ ]:
# Function for calculating Swirling Strength across the 3D domain 
def swirl_loc(vx, vy, vz, fname): 
    
    # Store gradient of the vector field (division by 10**6 is for unit correction)
    grad_v = np.array([np.gradient(vx),np.gradient(vy),np.gradient(vz)])/10**6

    # Delete unwanted variables
    del vz, vx, vy

    # Calculate eigenvalues at each point in the grid
    # Transpose is required in order to take the shape of the array correctly (3x3 matrix for each point)
    EV1 = np.linalg.eigvals(grad_v.transpose())
    
    del grad_v
    
    # Store coordinates of locations with Complex eigenvalues 
    loc = np.where(np.sum(np.abs(EV1.imag), axis = 3) != 0)
    
    # Magnitude of imaginary part of eigenvalue gives the swirling strength (SS) magnitude
    sw = np.sum(np.abs(EV1[loc].imag), axis=1)/2
    del EV1

    # Store location along with SS value in file
    loc_con = np.zeros((len(loc[0]),1))
    loc_con = np.insert(loc_con, 0, loc[0], axis=1)
    loc_con = np.insert(loc_con, 1, loc[1], axis=1)
    loc_con = np.insert(loc_con, 2, loc[2], axis=1)
    loc_con = np.insert(loc_con, 3, sw, axis=1)

    loc_con = np.delete(loc_con, 4, axis=1)

    np.save(fname, loc_con)
    del loc, sw, loc_con
    
# Change name of file accordingly    
#file = "LOC_full"
#swirl_loc(vx, vy, vz, file)

In [ ]:
# Compute eigenvectors and store real eigenvector, which acts as the axis of rotation
# It was necessary to divide this into 4 parts due to memory issues

def eigvect(vx, vy, vz):

    grad_v = np.array([np.gradient(vx),np.gradient(vy),np.gradient(vz)])/10**6 

    eigval, eigvect = np.linalg.eig(grad_v.transpose()[:, 0:600, 0:600, :, :])
    loc = np.where(np.sum(np.abs(eigval.imag), axis = 3) != 0)

    loc_con = np.zeros((len(loc[0]),1))
    loc_con = np.insert(loc_con, 0, loc[0], axis=1)
    loc_con = np.insert(loc_con, 1, loc[1], axis=1)
    loc_con = np.insert(loc_con, 2, loc[2], axis=1)
    loc_con = np.delete(loc_con, 3, axis=1)
    
    del eigval

    # Locations where determinant of matrix formed by eigenvectors have a negative imaginary part
    loc2 = np.where(np.linalg.det(eigvect[loc]).imag < 0 )
    
    # Location of real eigenvectors
    real = np.where(np.sum(np.abs(eigvect[loc].imag), axis = 1) == 0)
    sw_dir = eigvect[loc].transpose(0,2,1)[real[0], real[1]]
    del loc, real
    
    # This part is necessary to keep the handed-ness of the eigenvector consistent
    sw_dir[loc2] = sw_dir[loc2]*(-1)
    np.save("Eigvect", sw_dir.real)
    del eigvect


    eigval, eigvect = np.linalg.eig(grad_v.transpose()[:, 0:600, 600:1200, :, :])
    loc = np.where(np.sum(np.abs(eigval.imag), axis = 3) != 0)

    loc_con1 = np.zeros((len(loc[0]),1))
    loc_con1 = np.insert(loc_con1, 0, loc[0], axis=1)
    loc_con1 = np.insert(loc_con1, 1, loc[1], axis=1)
    loc_con1 = np.insert(loc_con1, 2, loc[2]+600, axis=1)
    loc_con1 = np.delete(loc_con1, 3, axis=1)
    
    del eigval

    loc2 = np.where(np.linalg.det(eigvect[loc]).imag < 0 )
    real = np.where(np.sum(np.abs(eigvect[loc].imag), axis = 1) == 0)
    sw_dir = eigvect[loc].transpose(0,2,1)[real[0], real[1]]
    del loc, real
    sw_dir[loc2] = sw_dir[loc2]*(-1)
    np.save("Eigvect1", sw_dir.real)
    del eigvect


    eigval, eigvect = np.linalg.eig(grad_v.transpose()[:, 600:1200, 0:600, :, :])
    loc = np.where(np.sum(np.abs(eigval.imag), axis = 3) != 0)

    loc_con2 = np.zeros((len(loc[0]),1))
    loc_con2 = np.insert(loc_con2, 0, loc[0], axis=1)
    loc_con2 = np.insert(loc_con2, 1, loc[1]+600, axis=1)
    loc_con2 = np.insert(loc_con2, 2, loc[2], axis=1)
    loc_con2 = np.delete(loc_con2, 3, axis=1)
    
    del eigval

    loc2 = np.where(np.linalg.det(eigvect[loc]).imag < 0 )
    real = np.where(np.sum(np.abs(eigvect[loc].imag), axis = 1) == 0)
    sw_dir = eigvect[loc].transpose(0,2,1)[real[0], real[1]]
    del loc, real
    sw_dir[loc2] = sw_dir[loc2]*(-1)
    np.save("Eigvect2", sw_dir.real)
    del eigvect


    eigval, eigvect = np.linalg.eig(grad_v.transpose()[:, 600:1200, 600:1200, :, :])
    loc = np.where(np.sum(np.abs(eigval.imag), axis = 3) != 0)

    loc_con3 = np.zeros((len(loc[0]),1))
    loc_con3 = np.insert(loc_con3, 0, loc[0], axis=1)
    loc_con3 = np.insert(loc_con3, 1, loc[1]+600, axis=1)
    loc_con3 = np.insert(loc_con3, 2, loc[2]+600, axis=1)
    loc_con3 = np.delete(loc_con3, 3, axis=1)

    del eigval


    loc2 = np.where(np.linalg.det(eigvect[loc]).imag < 0 )
    real = np.where(np.sum(np.abs(eigvect[loc].imag), axis = 1) == 0)
    sw_dir = eigvect[loc].transpose(0,2,1)[real[0], real[1]]
    del loc, real
    sw_dir[loc2] = sw_dir[loc2]*(-1)
    np.save("Eigvect3", sw_dir.real)
    del eigvect

    np.save("loc_con", loc_con)
    np.save("loc_con1", loc_con1)
    np.save("loc_con2", loc_con2)
    np.save("loc_con3", loc_con3)
    
# Concatenate all files together    

# eigvect(vx, vy, vz)
# eig = np.vstack((np.load("Eigvect.npy"), np.load("Eigvect1.npy"), np.load("Eigvect2.npy"), np.load("Eigvect3.npy")))
# LOC = np.vstack((np.load("loc_con.npy"), np.load("loc_con1.npy"), np.load("loc_con2.npy"), np.load("loc_con3.npy")))
# np.save("eigvect_full", eig)
# np.save("LOC_eigvect", LOC)

# LOC = np.load("LOC_eigvect.npy")
# eigvect = np.load("eigvect_full.npy")

In [ ]:
# Create file with swirling strength in the standard grid format with appropriate locations (1200 x 1200 x 400) 

# Change file name as is required
file1 = "LOC_full.npy"
file2 = "full_swirl.bin"

def swirl_grid(file1, file2)

    LOC_final = np.load(file1)
    full_swirl = np.zeros((1200,1200,400))
    full_swirl[(LOC_final[:,2].astype(int), LOC_final[:,1].astype(int), LOC_final[:,0].astype(int))] = LOC_final[:,3]
    full_swirl.astype(np.float32).tofile(file2)
    
# swirl_grid(file1, file2)

In [ ]:
# Store x, y, and z components of swirling strength vector in grid format

# File containing eigenvector locations
file1 = "LOC_eigvect.npy"

# File containing corresponding components of unit eigenvectors 
file2 = "eigvect_full.npy"

# File containing magnitude of swirling strength
file3 = "full_swirl.bin"


def swirl_vect(file1, file2, file3)

    LOC = np.load(file1)
    eigvect = np.load(file2)
    full_swirl = np.fromfile(file3, dtype=np.float32).reshape(nx,ny,nz)

    sw_x = np.zeros((1200,1200,400))
    sw_y = np.zeros((1200,1200,400))
    sw_z = np.zeros((1200,1200,400))

    sw_x[LOC[:,2].astype(int), LOC[:,1].astype(int), LOC[:,0].astype(int)] = eigvect[:,0]*full_swirl[LOC[:,2].astype(int), LOC[:,1].astype(int), LOC[:,0].astype(int)]
    sw_y[LOC[:,2].astype(int), LOC[:,1].astype(int), LOC[:,0].astype(int)] = eigvect[:,1]*full_swirl[LOC[:,2].astype(int), LOC[:,1].astype(int), LOC[:,0].astype(int)]
    sw_z[LOC[:,2].astype(int), LOC[:,1].astype(int), LOC[:,0].astype(int)] = eigvect[:,2]*full_swirl[LOC[:,2].astype(int), LOC[:,1].astype(int), LOC[:,0].astype(int)]

    sw_x.astype(np.float32).tofile('sw_x.bin')
    sw_y.astype(np.float32).tofile('sw_y.bin')
    sw_z.astype(np.float32).tofile('sw_z.bin')
    
# swirl_vect(file1, file2)

In [ ]:
# Selection of high swirling regions, plane-wise

def vort_plane(loc, z):
    
    # loc contains coordinates of Swirling Strength (SS) locations and its value
    
    # Choose only those locations at a height = z
    ind = np.where(loc[:,0]==z)
    plane_z = np.float32(loc[ind])
    
    # Delete SS value axis, so now plane_z contains only coordinates
    plane_z = np.delete(plane_z, 3, axis=1)
    
    # Store SS value separately (Log-scaled because the histogram then follows nearly normal distribution.)
    swirl_str = np.log10(loc[ind][:,3])
    
    # Uncomment the following lines if histogram needs to be plotted
    
    #plt.hist(swirl_str, bins=30, color='orange')
    #plt.ylabel('Counts')
    #plt.xlabel('Log$_{10}$ [Swirling Strength]')
    #plt.show()
    
    
    # Strong criteria where locations greater than 2 std of SS only are chosen (Modify as per choice)
    strong_vort = swirl_str[swirl_str > (np.mean(swirl_str) + 2*np.std(swirl_str))]
    #bin_strong[swirl_str > mean + 2*threshold] = 1
    
    # A slightly relaxed criteria (std > 1.5)
    avg_vort = swirl_str[swirl_str > (np.mean(swirl_str) + 1.5*np.std(swirl_str))]
    
    
    loc_strong = plane_z[swirl_str > (np.mean(swirl_str) + 2*np.std(swirl_str))]
    loc_avg = plane_z[swirl_str > (np.mean(swirl_str) + 1.5*np.std(swirl_str))]

    # Finally return strong and avg SS location coordinates along with log-scaled SS value
    loc_strong = np.insert(loc_strong, 3, strong_vort, axis=1)
    loc_avg = np.insert(loc_avg, 3, avg_vort, axis=1)
    
    return(loc_strong, loc_avg)


# z = 150
# loc = np.load("LOC_full.npy")
# loc_str, loc_avg = vort_plane(loc, z)

In [ ]:
# Storing locations in all planes where SS > than the set threshold

file1 = "LOC_full.npy"

# To denote velocity vortices
file2 = "Velocity"

def full_grid(file1, file2):

    LOC_final = np.load(file1)
    L_strong = np.zeros((1,4))
    L_avg = np.zeros((1,4))
    for i in range(400):

        l_str, l_avg = vort_plane(LOC_final, i)
        L_strong = np.insert(L_strong, len(L_strong), l_str, axis=0)
        L_avg = np.insert(L_avg, len(L_avg), l_avg, axis=0)

    L_strong = np.delete(L_strong, 0, axis=0)
    L_avg = np.delete(L_avg, 0, axis=0)
    np.save(file2 + "_Strong_vort", L_strong)
    np.save(file2 + "_Avg_vort", L_avg)
    
# full_grid(file1, file2)

In [ ]:
# Curl

def curl(u,v,w):
    
    dummy, dFx_dy, dFx_dz = np.gradient(u)
    dFy_dx, dummy, dFy_dz = np.gradient(v)
    dFz_dx, dFz_dy, dummy = np.gradient(w)

    rot_x = (dFz_dy - dFy_dz)/(10**6)
    rot_y = (dFx_dz - dFz_dx)/(10**6)
    rot_z = (dFy_dx - dFx_dy)/(10**6)

    l = np.sqrt(np.power(u,2.0) + np.power(v,2.0) + np.power(w,2.0));

    m1 = np.multiply(rot_x,u)
    m2 = np.multiply(rot_y,v)
    m3 = np.multiply(rot_z,w)

    tmp1 = (m1 + m2 + m3)
    tmp2 = np.multiply(l,2.0)

    av = np.divide(tmp1, tmp2)

    return rot_x, rot_y, rot_z, av


wx, wy, wz, av = curl(vx, vy, vz)

jx, jy, jz, av2 = curl(bx, by, bz)

wx.astype(np.float32).tofile('curl_x.bin')
wy.astype(np.float32).tofile('curl_y.bin')
wz.astype(np.float32).tofile('curl_z.bin')

jx.astype(np.float32).tofile('mcurl_x.bin')
jy.astype(np.float32).tofile('mcurl_y.bin')
jz.astype(np.float32).tofile('mcurl_z.bin')

In [ ]:
# Poynting Flux

vb = np.cross(np.array([vx, vy, vz]), np.array([bx, by, bz]), axisa=0, axisb=0)
vb_x = vb[:,:,:,0]
vb_y = vb[:,:,:,1]
vb_z = vb[:,:,:,2]


s = -np.cross(vb, np.array([bx, by, bz]), axisa=-1, axisb=0)/(4*np.pi)
sx = s[:,:,:,0]
sy = s[:,:,:,1]
sz = s[:,:,:,2]

sx.astype(np.float32).tofile('Sx.bin')
sy.astype(np.float32).tofile('Sy.bin')
sz.astype(np.float32).tofile('Sz.bin')